# Introduction

Welcome! The purpose of this Jupyter notebook is to illustrate the use of freely available software to relate crime data and spatial mobility data. Our focus for now is on **data preparation**. High quality data preparation is the foundation of any successful analysis or visualization. In practice, data preparation occupies most of the time devoted to an analytical project. 

In other parts of the workshop, we'll learn more about how to analyze and visualize our data. 

# Getting Started

We'll need a few `python` packages to get us started. You can install all of them using `pip`, which is likely included with your installation of ``python``. To install a package, open a terminal and type `pip install <package_name>`; for example, 

    pip install pandas

A few notes about our tools for today: 

- `pandas` is a tool for working with tables of data, also called `data frames`.
- `fiona` is a utility for reading and writing files containing spatial data. 
- `shapely` is a tool for manipulating spatial objects, like points and polygons.
- `rtree` is a performance-enhancing tool for relating spatial objects to each other. We don't strictly need it for this exercise, but it will help our code run much faster. 
- `numpy` is a module for a wide variety of tasks in scientific computing. 
- `seaborn` is a module for modern statistical visualization.

In [1]:
import pandas as pd
import fiona
from shapely.geometry import MultiPolygon, Point, shape
from rtree import index
import numpy as np

%matplotlib inline 

Let's read in our data and inspect it. Note that the data frame structure makes it easy to see rows and columns. 

In [2]:
crime = pd.read_csv('PVD_CrimeData_LatLng.csv')

IOError: File PVD_CrimeData_LatLng.csv does not exist

Let's rename the Lat2 and Lon2 columns to be a little more natural: 

In [ ]:
crime['lat'] = crime.Lat2
crime['lon'] = crime.Lng2

Let's check and see that we got what we expect...

In [ ]:
crime[['lat', 'lon']].head()

Woops! Looks like the first few rows don't actually have any spatial data. We'll discard these from analysis:

In [ ]:
crime = crime[np.isnan(crime.lat) == False]

In [ ]:
crime[['lat', 'lon']].head()

Looks good! Note that we lost ~150 rows out of a total of ~190,000 -- less than 0.1%. 

# Reading a Shapefile

Let's move on to the mobility data. Our mobility data is keyed to a shapefile of *tracts*. E.g., we know how many individuals move from tract *O* to tract *D*. So, our first task is to join the `crime` table to the shapefile that our mobility data reflects. 

We're going to use the ``fiona`` package to open up a shapefile containing census tracts for the city of Providence, and we'll use the ``shapely`` package to transform it into polygons that we can work with. 

In [ ]:
tracts_raw = fiona.open('PVD/PVD.shp')
tract_names = [tract['properties']['GEOID'] for tract in tracts_raw] # make a list of names for later
tracts = MultiPolygon([shape(tract['geometry']) for tract in tracts_raw])

A nice feature of the ``shapely`` module is that it makes it easy to take a peek at what you're working on: just type the name of your shape when working in a Jupyter notebook, and you'll get a little preview of your object. It's safe to ignore the warning in this case. 

It's a little scrunched up, but this is Providence County, containing all of the city of Providence. 

# Joining the data

Ok, we're read to conduct the join! This is the most technical part of today's exercise. The reason is that relating two spatial data sets is often **very** computationally expensive. There are two main challenges: 

1. Figuring out whether a point is in a polygon is hard for computers. 
2. We have a lot of points and a lot of polygons. 

To think about that first problem, imagine that I give you a point: 

In [ ]:
crime[['lon', 'lat']].tail(1)

And a single polygon: 

In [ ]:
tracts[0].to_wkt()

How fast could you tell whether or not the point was in the polygon? Fortunately, your computer can do this quickly -- but that leads us to our second problem. Keep in mind that we have about 1.9e5 rows in our `crime` table and 141 tracts: 

In [ ]:
len(crime), len(tracts)

To relate these two data sets, we would have to perform roughly

$$1.9\text{e}5 \times 141 \approx 2.6e7$$

comparisons between points and polygons. As we've seen, each of these comparisons is kind of complicated! Doing 26 million of them would take a while. 

Fortunately, we can use a **spatial index** to speed things up. This won't reduce the number of operations we need to do, but it will make each individual operation much, much faster. As a result, our code can run in minutes rather than days. 

First we'll build the index: 

In [ ]:
idx = index.Index()

# place the bounding box of each tract inside the index. 
for i in range(len(tracts)):
    idx.insert(i, tracts[i].bounds)

Next we'll define a function for querying the index. For each i, this function will construct the spatial point corresponding to the ith row in the `crime` table, and then return the name of the tract in which that point lies. 

In [ ]:
def get_tract(i):
    p = Point(crime.lon.iloc[i], crime.lat.iloc[i])
    for j in idx.intersection((p.x, p.y)):
        if p.within(tracts[j]):
            return tract_names[j]
    return None

Now we're ready! We'll make a new column in our `crime` table with the tract names. Note that this line will take a few minutes to run. 

In [ ]:
crime['tract'] = [get_tract(i) for i in range(len(crime))]

We now have the tract corresponding to each entry of the `crime` table: 

In [ ]:
crime.tract.head()

Great! We are ready to conduct analysis that incorporates both the crime table we've been working on and the mobility data of origin-destination trips. Before we move on, you might want to save your results in a new .csv file. 

In [ ]:
crime.to_csv('crime_joined.csv')

Great! Coming up, we'll use our joined data to construct mobility networks incorporating crime data. 